![](header.jpg)

# Global Positioning System

Kevin J. Walchko, Phd

4 July 2020

---

GPS, originally developed for the Department of Defense, is used for a variety of applications ranging from navigation to guiding military munitions to a taget.

![](https://cdn-learn.adafruit.com/assets/assets/000/031/843/medium640/gps_746_iso.jpg?1461187948)

Using Adafruit's gps library for CircuitPython on normal python, I can parse the GPS strings.

- Satellites: 22 tracking, 66 searching
- Patch Antenna Size: 15mm x 15mm x 4mm
- Update rate: 1 to 10 Hz
- Position Accuracy: < 3 meters (all GPS technology has about 3m accuracy)
- Velocity Accuracy: 0.1 meters/s
- Warm/cold start: 34 seconds
- Acquisition sensitivity: -145 dBm
- Tracking sensitivity: -165 dBm
- Maximum Velocity: 515m/s
- Vin range: 3.0-5.5VDC
- MTK3339 Operating current: 25mA tracking, 20 mA current draw during navigation
- Output: NMEA 0183, 9600 baud default, 3V logic level out, 5V-safe input
- DGPS/WAAS/EGNOS supported
- FCC E911 compliance and AGPS support (Offline mode : EPO valid up to 14 days )
- Up to 210 PRN channels
- Jammer detection and reduction
- Multi-path detection and compensation


## References

- [Adafruit Ultimate GPS Breakout - Version 3](https://www.adafruit.com/product/746)
- [Datasheet](GlobalTop-FGPMMOPA6H-Datasheet-V0A.pdf)
- [NMEA0183 protocol reference](NMEA0183.pdf)
- [Datasheet](PMTK_A11.pdf)

In [31]:
import serial
import time
from adafruit_gps import GPS
import attr
from collections import namedtuple

In [32]:
Geolocation = namedtuple("Geolocation", "lat lon alt speed timestamp")
GpsFix = namedtuple("GpsFix", "fix fix_quality fix_3d num_sats")

In [33]:
# macOS
port = "/dev/tty.usbserial-A506BOT5"

# Linux
# port = "/dev/serial/by-id/usb-FTDI_FT232R_USB_UART_A506BOT5-if00-port0"

In [45]:
ser = serial.Serial(port, baudrate=9600, timeout=3000)
gps = GPS(ser)

"""
PMTK_A11.pdf pg 12
Data Field：
There are totally 19 data fields that present output frequencies for the 19 supported
NMEA sentences individually.
Supported NMEA Sentences
0 NMEA_SEN_GLL, // GPGLL interval - Geographic Position - Latitude longitude
1 NMEA_SEN_RMC, // GPRMC interval - Recommended Minimum Specific GNSS Sentence
2 NMEA_SEN_VTG, // GPVTG interval - Course over Ground and Ground Speed
3 NMEA_SEN_GGA, // GPGGA interval - GPS Fix Data
4 NMEA_SEN_GSA, // GPGSA interval - GNSS DOPS and Active Satellites
5 NMEA_SEN_GSV, // GPGSV interval - GNSS Satellites in View
6 //Reserved
7 //Reserved
13 //Reserved
14 //Reserved
15 //Reserved
16 //Reserved
17 //Reserved
18 NMEA_SEN_MCHN, // PMTKCHN interval – GPS channel status
Supported Frequency Setting
0 - Disabled or not supported sentence
1 - Output once every one position fix
2 - Output once every two position fixes
3 - Output once every three position fixes
4 - Output once every four position fixes
5 - Output once every five position fixes
Example:
$PMTK314,1,1,1,1,1,5,0,0,0,0,0,0,0,0,0,0,0,0,0*2C<CR><LF>
"""
# gps.send_command(b'PMTK314,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0') # minimum
gps.send_command(b'PMTK314,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0') # setup typical
# gps.send_command(b'PMTK314,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0') # everything
# gps.send_command(b'PMTK220,1000')  # 1 Hz
gps.send_command(b'PMTK220,500')  # 2 Hz, 500 msec

for i in range(3):
    # data = ser.read(1024)
    # print(data, flush=True)
    time.sleep(1.0)

    ok = gps.update()
    if not ok or gps.has_fix is False:
        print("no data, fix:", gps.has_fix)
    else:
        print('=' * 40)  # Print a separator line.
        print("Fix:", gps.has_fix)
        print("Satellites:", gps.satellites)
        print("Sats:", gps.sats)
#         print('Latitude: {0:.6f} degrees'.format(gps.latitude))
#         print('Longitude: {0:.6f} degrees'.format(gps.longitude))
        print("Alititude [m]:", gps.altitude_m)
        print("Altitude Geod [m]", gps.height_geoid)
        print("Speed [knots]:", gps.speed_knots)
        print("Dop: {} {} {}".format(gps.pdop, gps.hdop, gps.vdop))
        print("Fix Quality: {} {}".format(gps.fix_quality, gps.fix_quality_3d))
        print("PRNs:", gps.sat_prns)
        # print("Fix Raw timestamp:", gps.timestamp_utc)
        print('Fix timestamp: {:02}/{:02}/{} {:02}:{:02}:{:02}'.format(
            gps.timestamp_utc.tm_mon,   # Grab parts of the time from the
            gps.timestamp_utc.tm_mday,  # struct_time object that holds
            gps.timestamp_utc.tm_year,  # the fix time.  Note you might
            gps.timestamp_utc.tm_hour,  # not get all data like year, day,
            gps.timestamp_utc.tm_min,   # month!
            gps.timestamp_utc.tm_sec))


no data, fix: False
Fix: True
Satellites: 4
Sats: None
Alititude [m]: 56.4
Altitude Geod [m] -33.5
Speed [knots]: None
Dop: None None None
Fix Quality: 1 None
PRNs: None
Fix timestamp: 00/00/0 17:00:49
Fix: True
Satellites: 4
Sats: None
Alititude [m]: 56.4
Altitude Geod [m] -33.5
Speed [knots]: 0.07
Dop: None None None
Fix Quality: 1 None
PRNs: None
Fix timestamp: 07/05/2020 17:00:49


In [44]:
data = []

for i in range(10):
    # data = ser.read(1024)
    # print(data, flush=True)
    time.sleep(0.5)

    ok = gps.update()
    if not ok or gps.has_fix is False:
        print("no data")
    else:
        f = GpsFix(
            gps.has_fix, gps.fix_quality, gps.fix_quality_3d, gps.satellites)
        d = Geolocation(
            gps.latitude, gps.longitude, gps.altitude_m, 
            gps.speed_knots, time.monotonic)
        data.append(d)
#         print(d)
        print(f)

GpsFix(fix=True, fix_quality=1, fix_3d=None, num_sats=4)
GpsFix(fix=True, fix_quality=1, fix_3d=None, num_sats=4)
GpsFix(fix=True, fix_quality=1, fix_3d=None, num_sats=4)
GpsFix(fix=True, fix_quality=1, fix_3d=None, num_sats=4)
GpsFix(fix=True, fix_quality=1, fix_3d=None, num_sats=4)
GpsFix(fix=True, fix_quality=1, fix_3d=None, num_sats=4)
GpsFix(fix=True, fix_quality=1, fix_3d=None, num_sats=4)
GpsFix(fix=True, fix_quality=1, fix_3d=None, num_sats=4)
GpsFix(fix=True, fix_quality=1, fix_3d=None, num_sats=4)
GpsFix(fix=True, fix_quality=1, fix_3d=None, num_sats=4)
